In [1]:
from tensorflow.keras import datasets, layers, models

import cv2 as cv
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

In [2]:
def padImage(image, pixels=5):
    bottom = image[-pixels:]
    top = image[:pixels]

    img = np.insert(image, 0, bottom, 0)
    img = np.insert(img, len(img), top, 0)
    img = np.insert(img, [0], [0] * pixels, 1)
    img = np.insert(img, [-1], [0] * pixels, 1)
    return img

In [3]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = (train_images / 255.0).astype(np.float32), test_images.astype(np.float32) / 255.0

In [4]:
### THE PAD HAS TO BE DONE IN THE
### POLAR SPACE

# 20 is the ceiling of (14 * sqrt(2))
X_train_polar = [cv.linearPolar(x, tuple(np.array(x.shape)/2), 20, cv.WARP_FILL_OUTLIERS) for x in train_images]
X_train_polar = [padImage(x, pixels=5) for x in X_train_polar]
X_train_polar = np.array(X_train_polar)[...,None]

X_test_polar = [cv.linearPolar(x, tuple(np.array(x.shape)/2), 20, cv.WARP_FILL_OUTLIERS) for x in test_images]
X_test_polar = [padImage(x, pixels=5) for x in X_test_polar]
X_test_polar = np.array(X_test_polar)[...,None]

# Rotate test set
X_test_r_polar = [tfa.image.rotate(x, np.random.uniform(-np.pi/2., np.pi/2.)).numpy() for x in test_images]
X_test_r_polar = [cv.linearPolar(x, tuple(np.array(x.shape)/2), 20, cv.WARP_FILL_OUTLIERS) for x in X_test_r_polar]
X_test_r_polar = [padImage(x, pixels=5) for x in X_test_r_polar]
X_test_r_polar = np.array(X_test_r_polar)[...,None]

In [28]:
model = models.Sequential()
model.add(layers.Input(shape=X_train_polar.shape[1:]))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=True))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=True))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))


model.add(layers.GlobalMaxPooling2D())
model.add(layers.LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=True))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [29]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(X_train_polar, train_labels, batch_size=32, epochs=10, 
          validation_data=(X_test_r_polar, test_labels),
          callbacks=[reduce_lr])

Epoch 1/10
1875/1875 [==============================] - 99s 52ms/step - loss: 1.0635 - accuracy: 0.6302 - val_loss: 0.4357 - val_accuracy: 0.8660
Epoch 2/10
1875/1875 [==============================] - 107s 57ms/step - loss: 0.1954 - accuracy: 0.9389 - val_loss: 0.4668 - val_accuracy: 0.8486
Epoch 3/10
1875/1875 [==============================] - 108s 58ms/step - loss: 0.1471 - accuracy: 0.9547 - val_loss: 0.3880 - val_accuracy: 0.8685
Epoch 4/10
1875/1875 [==============================] - 107s 57ms/step - loss: 0.1223 - accuracy: 0.9623 - val_loss: 0.4141 - val_accuracy: 0.8668
Epoch 5/10
1875/1875 [==============================] - 108s 57ms/step - loss: 0.1111 - accuracy: 0.9660 - val_loss: 0.3754 - val_accuracy: 0.8815
Epoch 6/10
1875/1875 [==============================] - 107s 57ms/step - loss: 0.1031 - accuracy: 0.9679 - val_loss: 0.3487 - val_accuracy: 0.8869
Epoch 7/10
1875/1875 [==============================] - 107s 57ms/step - loss: 0.0989 - accuracy: 0.9701 - val_loss: 0.

In [10]:
# Plain accuracy
accuracy = np.mean(model.predict(X_test_polar).argmax(axis=1) == test_labels)*100
# "Rotated" accuracy
accuracy_rotated = np.mean(model.predict(X_test_r_polar).argmax(axis=1) == test_labels)*100

print(accuracy, accuracy_rotated)

59.95 41.4
